In [1]:
import pandas as pd
import numpy as np
import os
import pyBigWig

In [2]:
active_SDRCI = pd.read_csv('temp/active_SDRCI.csv')
repress_SDRCI = pd.read_csv('temp/repress_SDRCI.csv')

active_loop = '/home/lmh/work/repos/MINE/analyse/fig3/tss/temp/HepG2_ATAC_H3K27ac_H3K4me3_2_100_all_enhanced_sig.npy'
repress_loop = '/home/lmh/work/repos/MINE/analyse/fig3/represses/tss/temp/HepG2_H3K9me3_H3K27me3_2_100_all_enhanced_sig.npy'

CTCF = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/CTCF.bigWig')
# CEBPB = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/CEBPB.bigWig')
POLR2A = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/POLR2A.bigWig')
EZH2 = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/EZH2.bigWig')
# KDM4B = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/KDM4B.bigWig')
# H3K27ac = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/H3K27ac.bigWig')
H3K27me3 = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/H3K27me3.bigWig')
H3K9me3 = pyBigWig.open('/data1/lmh_data/MINE/source/HepG2/H3K9me3.bigWig')

In [3]:
def cal_tad_enrich_quality(SDRCI_info, bw):
    result = {
        'low': [],
        'middle': [],
        'high': [],
        'ultra high': []
    }
    for i, row in SDRCI_info.iterrows():
        signal = bw.stats(row['0'], row['1'], row['2'], type="mean")
        result[row['degree']].append(signal[0])
    for key in result:
        result[key] = np.mean(result[key])
    return result

In [4]:
for peak in [CTCF, POLR2A, EZH2]:
    print(cal_tad_enrich_quality(active_SDRCI, peak))

{'low': 0.5283164324261734, 'middle': 0.599167683430085, 'high': 0.6492088633307508, 'ultra high': 0.738597187660738}
{'low': 0.6138141157886398, 'middle': 0.9700560515005915, 'high': 1.4629083336574216, 'ultra high': 1.9327257494097434}
{'low': 0.5723584128704784, 'middle': 0.6018062319718883, 'high': 0.5702928129953002, 'ultra high': 0.43733102684649894}


In [5]:
for peak in [CTCF, POLR2A, EZH2]:
    print(cal_tad_enrich_quality(repress_SDRCI, peak))

{'low': 0.5179242885021589, 'middle': 0.5502496385871951, 'high': 0.5742143322350551, 'ultra high': 0.6332065324905196}
{'low': 0.5627887152953883, 'middle': 0.6974916836607652, 'high': 0.8525353329483084, 'ultra high': 0.8010868486504602}
{'low': 0.6230545689915904, 'middle': 0.7682705173755787, 'high': 1.2634530237299695, 'ultra high': 2.150585815264372}


In [6]:
def read_loop_info(_path, chrom, start, end, FDR=0.5):
    loops = np.load(_path, allow_pickle=True).item()
    loops = loops[chrom]
    loops = list(filter(lambda x: ((x[0]>=start)&(x[0]<=end)|(x[1]>=start)&(x[1]<=end)), loops))
    return loops

def cal_tad_loop_enrich_quality(SDRCI_info, loop_info, bw):
    result = {
        'low': [],
        'middle': [],
        'high': [],
        'ultra high': []
    }
    for i, row in SDRCI_info.iterrows():
        loops = read_loop_info(loop_info, int(row['0'][3:]), row['1']/1000, row['2']/1000)
        
        for loop in loops:
            signal_0 = bw.stats(row['0'], int(loop[0]*1000), int((loop[0]+1)*1000), type="mean")
            signal_1 = bw.stats(row['0'], int(loop[1]*1000), int((loop[1]+1)*1000), type="mean")
            signal = np.mean([signal_0, signal_1])
        result[row['degree']].append(signal)
    for key in result:
        result[key] = np.mean(result[key])
    return result

In [7]:
for peak in [CTCF, POLR2A, EZH2]:
    print(cal_tad_loop_enrich_quality(active_SDRCI, active_loop, peak))

{'low': 1.809147853770284, 'middle': 2.079646424538973, 'high': 2.282993458090895, 'ultra high': 2.217899129224612}
{'low': 2.6529604710398496, 'middle': 3.416472331537373, 'high': 4.068648986842357, 'ultra high': 3.067326849013632}
{'low': 0.49247756928652847, 'middle': 0.7416338900488383, 'high': 0.6455730950466119, 'ultra high': 0.4468414944991722}


In [8]:
for peak in [H3K27me3, H3K9me3, EZH2]:
    print(cal_tad_loop_enrich_quality(repress_SDRCI, repress_loop, peak))

{'low': 3.2240465751996013, 'middle': 2.8280851336520536, 'high': 3.3624177767252195, 'ultra high': 3.715093834418804}
{'low': 5.2779899598885445, 'middle': 6.748040114621808, 'high': 5.96537557701515, 'ultra high': 5.624930463698908}
{'low': 8.772430314164065, 'middle': 5.635868269340967, 'high': 7.239905031254381, 'ultra high': 9.506174882743359}
